In [14]:
import os

import pandas as pd
import numpy as np



In [25]:
df = pd.read_parquet(r'../dataset/churn.parquet')
df.head(3)

,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,XW0DQ7H,Village,Platinum Membership,17-08-2017,No,xxxxxxxx,Gift Vouchers/Coupons,?,...,300.63,53005.25,17,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,32,F,5K0N3X1,City,Premium Membership,28-08-2017,?,CID21329,Gift Vouchers/Coupons,Desktop,...,306.34,12838.38,10,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,44,F,1F2TCL3,Town,No Membership,11-11-2016,Yes,CID12313,Gift Vouchers/Coupons,Desktop,...,516.16,21027.00,22,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1


In [ ]:
print('count: Records:', df.shape[0])
print('count: features:', df.shape[1], end = '\n\n')


In [ ]:
print(f'List of Features: {df.columns.values}')

In [ ]:
def list_feature_values(df):
    for col in df.columns:
        FeatureName = col
        Feature_Cardinality = df[col].nunique()
        Feature_Cardinality_Values = [
            df[col].unique() 
            if df[col].nunique() < 10 
            else f"{df[col].unique()[:10]}, ..."
        ]
        print('Feature:', FeatureName)
        print('count: Unique_Vals:', Feature_Cardinality)
        print('Unique_Vals:', Feature_Cardinality_Values, end = '\n\n')

        